# Raster aggregation

This notebook contains code for aggregating (resampling down) categorical rasters, for example to convert the MODIS 30 arcsecond (~1km) grids into 2.5 arcminute (~5km) grids. (Code for aggregating continuous rasters is provided in another notebook/module).

Categorical rasters can be aggregated to produce one class proportion and one like adjacency grid for each of the input values, and a single majority grid. This assumes that the input grids have a small-ish number of unique values, and are in unsigned 8 bit integer format (it was written for the MCD12Q1 BRDF landcover data).

The actual aggregation code is in the Cython_Raster_Funcs/RasterAggregator_Categorical.pxd file. This is, or needs to be, translated to C using the Cython translation / compilation tools. This notebook provides demonstrations of how to run the cythonized code.

The code has been written to read input rasters of theoreticlly unlimited size, which are read in tiles to build up the output coarser / smaller grids. 


In [1]:
from osgeo import gdal
import numpy as np

import glob
import os

### Import / build the aggregation function

In [2]:
# Import the cython module. If the c code has not already been generated by cython then you
# first need to change to the directory containing the .pxd file and run 
# python setup.py build_ext --inplace

from Cython_Raster_Funcs.RasterAggregator_Categorical import RasterAggregator_Categorical

### Import other common functions

In [3]:
from General_Raster_Funcs.GeotransformCalcs import calcAggregatedProperties
from General_Raster_Funcs.RasterTiling import getTiles
from General_Raster_Funcs.TiffManagement import SaveLZWTiff

## Code to run the aggregation function

This function demonstrates splitting the incoming file(s) into tiles if necessary, loading the data, running the cython-based aggregation function, and saving the results. It's based on configuring a few global variables in the notebook just for simplicity.

In [24]:
def categoricalAggregationRunner(dataPaths, categories):
    '''Run the aggregation code for each file specifed in dataPaths
    
    which should be a list of tiff files.
    
    The global notebook variables outDir, method, minMaxRangeSumOnly, itemsToSave, tileSize, 
    and (aggregationFactor OR resolution OR requiredXSize and requiredYSize)
    should be set first, along with the fnGetter function for producing output filenames.
    '''
    for f in dataPaths:
        print f
        ds = gdal.Open(f)
        b = ds.GetRasterBand(1)
        catNdv = b.GetNoDataValue()
        if catNdv is None:
            # force for water extent:
            catNdv = 255
            print "no ndv"
        else:
            catNdv = int(catNdv)
            print "incoming ndv is "+str(catNdv)
        fltNdv = -9999
        inputGT = ds.GetGeoTransform()
        inputProj = ds.GetProjection()
        
        if isinstance(categories, int):
            assert categories <= 256
            assert categories < 256
            assert categories >= 0
            nCategories = categories
        else:
            assert len(categories) <= 256
            assert max(categories) < 256
            assert min(categories) >= 0

            nCategories = len(categories)

        nBytesRequired = ds.RasterXSize * ds.RasterYSize * 1 * nCategories

        outGT, outShape = calcAggregatedProperties(method, (ds.RasterYSize, ds.RasterXSize), 
                                                   inputGT, aggregationFactor or None, 
                                                   (requiredYSize, requiredXSize), 
                                                   resolution or None)
        outYSize, outXSize = outShape    
        tiles = getTiles(ds.RasterXSize, ds.RasterYSize, tileSize)
        
        aggregator = RasterAggregator_Categorical(ds.RasterXSize, ds.RasterYSize, 
                                            outXSize, outYSize,
                                            nCategories,
                                            fltNdv,
                                            catNdv)
        print "Running {0!s} tiles".format(len(tiles)),
        for tile in tiles:
            print ".",
            xoff = tile[0][0]
            yoff = tile[1][0]
            xsize = tile[0][1] - xoff
            ysize = tile[1][1] - yoff
            inArr = b.ReadAsArray(xoff, yoff, xsize, ysize).astype(np.uint8)
            # if required, do a cheeky reclass along the way
            # e.g. to categorise into binary data before using that as the aggregation basis
            if reclassify is not None:
                inArr = reclassify(inArr)
            aggregator.addTile(inArr, xoff, yoff)
        r = aggregator.GetResults()
        itemstosave = ['fractions','likeadjacencies','majority']
        if r is None:
            raise Exception("An error occurred running the aggregation code")
        print r['valuemap']
        for i in range (0, nCategories):
            outValue = r['valuemap'][i]
            outRasterName = fnGetter(os.path.basename(f), outValue, "Percentage")
            SaveLZWTiff(r['fractions'][i], fltNdv, outGT, inputProj, outDir, outRasterName)
            outRasterName = fnGetter(os.path.basename(f), outValue, "LikeAdjacency")
            SaveLZWTiff(r['likeadjacencies'][i], fltNdv, outGT, inputProj, outDir, outRasterName)
            
        outRasterName = fnGetter(os.path.basename(f), "Overall", "Majority")
        SaveLZWTiff(r['majority'], catNdv, outGT, inputProj, outDir, outRasterName)

        

### Define the globals to configure the data we want to use in the aggregation runner

In [41]:
# All of the items in this cell are required by the continuousAggregationRunner function:

# 1. Specify the method by which the aggregation will be described
method = "resolution" # "factor" or "size" or "resolution"

# and whichever one of these is relevant
# if method = 'factor' then
aggregationFactor = None #5
# OR if method = 'size' then
requiredXSize = 8640 
requiredYSize = 4320
# OR if method = 'resolution' then specify a numeric cellsize or 
# a string from '1km', '5km', '10km'
resolution = '5km' #0.25

# 4. specify the folder where the outputs should be saved
#outDir = r"C:\temp\testagg\categorical"
outDir = r"C:\Temp\dataprep\water\seasonality"
# 5. specify a function called fnGetter, to get the output filename 
# (excluding folder), given an input filename and statistic type 
# (as in the itemstosave above)
outNameTemplate = r'{0!s}.{1!s}.{2!s}.tif'
fnGetter = lambda fn, num, tag:(outNameTemplate.format(
    os.path.splitext(os.path.basename(fn))[0],
    "Class_" + str(num),
    tag))
likeAdjNDV = -9999

# 6. specify a maximum tilesize for data to read in
# for the global 1km grids use 43200 to run untiled
tileSize = 43200

# 7. optionally specify a reclassify function to pre-prepare the incoming data into categories
reclassify = None

#def reclassify(arr):
#    import numexpr as ne
#    a = arr
#    expr = 'where (a < 80, 0, where (a== 255, 255, 1))'
#    return ne.evaluate(expr).astype(np.uint8)
    


### Define the paths of the data file(s)

In [36]:
# provide them as a list even if only 1
#dataPaths = [r'G:\MCD12Q1\500m\A2013001_MCD12Q1.tif']
dataPaths = [r'E:\Temp\water\seasonality.vrt']
#dataPaths = [r'C:\Users\zool1301.NDPH\Documents\Other_Data\GUF\GUF28\GUF28\00116666665417_N\0011_N_files_aligned.tif']
#dataPaths = glob.glob(r'C:\Users\zool1301.NDPH\Documents\Other_Data\GUF\GUF28\GUF28\**\*files_aligned.tif')
# generate with glob pattern if there's lots e.g. 
# dataPaths = glob.glob(r'G:\MCD12Q1\500m\A2013*.tif')

In [ ]:
dataPaths

### Run the code

You need to provide a list of category values or just the number of categories that there are. This should be a small-ish number, depending on the size of the output grids requested: there will be one like-adjacency and one fraction grid to fit into memory for each category. In any case there must be <= 255 categories.

In [ ]:
categoricalAggregationRunner(dataPaths, 12)

#### The fnGetter function could be more sophisticated to name the outputs more descriptively

In [17]:
# The values that the input raster has, which should start from zero
ibgpCats = {
    0:'Water',
    1:'Evergreen_Needleleaf_Forest',
    2:'Evergreen_Broadleaf_Forest',
    3:'Deciduous_Needleleaf_Forest',
    4:'Deciduous_Broadleaf_Forest',
    5:'Mixed_Forest',
    6:'Closed_Shrublands',
    7:'Open_Shrublands',
    8:'Woody_Savannas',
    9:'Savannas',
    10:'Grasslands',
    11:'Permanent_Wetlands',
    12:'Croplands',
    13:'Urban_And_Built_Up',
    14:'Cropland_Natural_Vegetation_Mosaic',
    15:'Snow_And_Ice',
    16:'Barren_Or_Sparsely_Populated',
    17:'Unclassified', # =254 in source data, hardcoded a remap in the cython
    18:'No_Data' # =255 in source data
}

fnGetter = lambda fn, num, tag:(outNameTemplate.format(
    os.path.splitext(os.path.basename(fn))[0],
    "Class_" + str(num) + "_" + ibgpCats[num] + "_",
    tag))